# 03 - Spark Engine Preview (Phase 1 Scaffold)

## 🧭 Goal

Understand ODIBI's Spark engine architecture and configuration.

This notebook will:
- Explore the Spark engine scaffold (Phase 1)
- Show Spark pipeline YAML structure
- Explain Azure integration points
- Preview what's coming in Phase 3

**Note:** Spark execution is coming in Phase 3 (Q2 2026). This shows the architecture.

**Estimated time:** 3 minutes

## 🔧 Setup

In [ ]:
# ✅ Environment Setup
import sys
import os
from pathlib import Path
import yaml

# Navigate to project root
project_root = Path.cwd().parent if Path.cwd().name == 'walkthroughs' else Path.cwd()
os.chdir(project_root)

print(f"✅ Environment ready")
print(f"📁 Working directory: {Path.cwd()}")

## 🏗️ Spark Engine Architecture

ODIBI's Spark engine follows the same interface as Pandas:

In [ ]:
# Inspect Spark engine code
with open('odibi/engine/spark_engine.py', 'r') as f:
    content = f.read()

print("📦 Spark Engine Class Structure:\n")

# Show class definition and key methods
lines = content.split('\n')
for i, line in enumerate(lines[:50], 1):
    if 'class SparkEngine' in line or 'def ' in line:
        print(f"{i:3}: {line}")

In [ ]:
# Try importing SparkEngine (will show helpful error if pyspark not installed)
try:
    from odibi.engine.spark_engine import SparkEngine
    print("✅ SparkEngine imported successfully")
    print(f"   Engine name: {SparkEngine.name}")
except ImportError as e:
    print(f"⚠️ Expected: {e}")
    print("\n💡 This is normal! Spark engine requires:")
    print("   pip install 'odibi[spark]'")

## 📋 Spark Pipeline Configuration

Let's examine the `example_spark.yaml` configuration:

In [ ]:
# Load Spark example configuration
with open('examples/example_spark.yaml', 'r') as f:
    spark_config = yaml.safe_load(f)

print("🔧 Spark Pipeline Configuration:\n")
print(f"Project: {spark_config['project']}")
print(f"Engine: {spark_config['engine']}")
print(f"\nConnections:")
for conn_name, conn_config in spark_config['connections'].items():
    print(f"  • {conn_name}: {conn_config['type']}")

In [ ]:
# Show Azure ADLS connection structure
print("\n☁️ Azure ADLS Connection:")
print(yaml.dump(spark_config['connections']['adls_bronze'], default_flow_style=False))

In [ ]:
# Show pipeline structure
print("\n📊 Pipeline Structure:")
for pipeline in spark_config['pipelines']:
    print(f"\nPipeline: {pipeline['name']}")
    print(f"  Nodes: {len(pipeline['nodes'])}")
    for node in pipeline['nodes']:
        print(f"    • {node['name']}")

## 🔗 Azure Connection Resolution

The Azure ADLS connection can build URIs even in Phase 1:

In [ ]:
# Test ADLS URI generation (Phase 1 feature that works now)
from odibi.connections.azure_adls import AzureADLS

# Create connection
adls = AzureADLS(
    account="mystorageaccount",
    container="datalake",
    path_prefix="bronze/",
    auth_mode="managed_identity"
)

# Build URI
uri = adls.uri("sensors/2024/01/data.parquet")

print("✅ ADLS URI Construction (Phase 1 Working):")
print(f"\n   Input path: sensors/2024/01/data.parquet")
print(f"   Output URI: {uri}")
print(f"\n   This URI will be used by Spark in Phase 3!")

## 🪞 Reflect

**What we learned:**
- Spark engine architecture (scaffolded in Phase 1)
- Spark pipeline YAML structure
- Azure ADLS connection configuration
- URI construction works today (Phase 1)
- Spark execution coming in Phase 3

**Phase 1 (Current) Features:**
- ✅ SparkEngine class defined
- ✅ Azure connections (ADLS, SQL, DBFS)
- ✅ Path/URI resolution
- ✅ Import guards with helpful errors

**Phase 3 (Q2 2026) Features:**
- ⏳ `SparkEngine.read()` - Load Parquet/CSV from ADLS
- ⏳ `SparkEngine.write()` - Save to ADLS/Delta
- ⏳ `SparkEngine.execute_sql()` - Run Spark SQL transforms
- ⏳ Integration tests with local Spark session

**Next step:**  
Go to **`04_ci_cd_and_precommit.ipynb`** to learn about code quality automation.

## ✅ Self-Check

In [ ]:
# ✅ Self-Check
try:
    import sys, os
    print("Running self-check...")
    
    # Verify Spark scaffolding exists
    assert os.path.exists("odibi/engine/spark_engine.py"), "Missing Spark engine"
    assert os.path.exists("odibi/connections/azure_adls.py"), "Missing ADLS connection"
    assert os.path.exists("examples/example_spark.yaml"), "Missing Spark example"
    
    # Verify Azure connections work
    from odibi.connections.azure_adls import AzureADLS
    conn = AzureADLS(account="test", container="data")
    uri = conn.uri("file.csv")
    assert uri.startswith("abfss://"), "URI generation failed"
    print(f"✅ Azure ADLS connection works")
    
    # Verify docs exist
    assert os.path.exists("docs/setup_databricks.md"), "Missing Databricks setup guide"
    assert os.path.exists("docs/setup_azure.md"), "Missing Azure setup guide"
    
    print("🎉 Walkthrough 03 verified successfully")
except Exception as e:
    print(f"❌ Walkthrough failed self-check: {e}")
    raise